# Progetto Marazzina — Gruppo 7
## Digital Put sotto GBM: Crank-Nicolson vs Eulero Implicito

**Payoff:** $\varphi(S) = \mathbb{1}_{S < K}$

**Dinamica GBM:** $dS = rS\,dt + \sigma S\,dW_t$

| Parametro | Valore |
|-----------|--------|
| r | 0.1% |
| σ | 0.4 |
| S₀, K | 1 |
| T | 2 anni |

## 1. Discretizzazione della PDE

La PDE di Black-Scholes, con il cambio $x = \ln(S/K)$ e $\tau = T - t$, diventa a **coefficienti costanti**:

$$\frac{\partial V}{\partial \tau} = \underbrace{\frac{\sigma^2}{2}}_{D} \frac{\partial^2 V}{\partial x^2} + \underbrace{\left(r - \frac{\sigma^2}{2}\right)}_{\nu} \frac{\partial V}{\partial x} - rV$$

Con differenze finite centrate su griglia uniforme definiamo:

$$\alpha = D\frac{\Delta\tau}{\Delta x^2}, \qquad \beta = \nu\frac{\Delta\tau}{2\Delta x}$$

Il $\theta$-schema è:

$$(I - \theta L)\,V^{n+1} = (I + (1-\theta)L)\,V^n + BC$$

dove $L$ è tridiagonale con diagonali $(\alpha-\beta,\;-2\alpha-r\Delta\tau,\;\alpha+\beta)$.

- **Eulero Implicito** ($\theta=1$): $O(\Delta\tau)$ in tempo, $O(\Delta x^2)$ in spazio.
- **Crank-Nicolson** ($\theta=0.5$): $O(\Delta\tau^2)$ in tempo, $O(\Delta x^2)$ in spazio.

**Condizioni al bordo** (digital put):
- $x \to -\infty$ (S→0, deep ITM): $V = e^{-r\tau}$
- $x \to +\infty$ (S→∞, deep OTM): $V = 0$

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

# Parametri
r     = 0.001
sigma = 0.4
S0    = 1.0
K     = 1.0
T     = 2.0
L_mult = 6  # griglia: x in [-L*sigma*sqrt(T), +L*sigma*sqrt(T)]

## 2. Soluzione analitica (benchmark)

$$V = e^{-rT} \,\Phi(-d_2), \quad d_2 = \frac{\ln(S_0/K) + (r - \sigma^2/2)T}{\sigma\sqrt{T}}$$

In [ ]:
d2 = (np.log(S0/K) + (r - 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
V_exact = np.exp(-r*T) * norm.cdf(-d2)
print(f"d2       = {d2:.6f}")
print(f"N(-d2)   = {norm.cdf(-d2):.6f}")
print(f"V_exact  = {V_exact:.10f}")

## 3. Solver PDE

In [ ]:
def solve_digital_put(r, sigma, K, T, M, N, theta):
    """
    Risolve la PDE in log-space x=ln(S/K) con theta-schema.
    theta=1.0 -> Eulero Implicito
    theta=0.5 -> Crank-Nicolson
    Restituisce (x_grid, V) con M+2 punti totali.
    """
    nu = r - 0.5 * sigma**2
    x_min = -L_mult * sigma * np.sqrt(T)
    x_max =  L_mult * sigma * np.sqrt(T)
    dx = (x_max - x_min) / (M + 1)
    dt = T / N
    x = np.linspace(x_min, x_max, M + 2)

    alpha = 0.5 * sigma**2 * dt / dx**2
    beta  = nu * dt / (2.0 * dx)
    r_dt  = r * dt

    # Costruzione matrici sparse tridiagonali (M x M, nodi interni)
    ones = np.ones(M)

    # LHS: (I - theta*L)
    A = sp.diags(
        [-theta * (alpha - beta) * ones[:-1],
         (1.0 + theta * (2.0*alpha + r_dt)) * ones,
         -theta * (alpha + beta) * ones[:-1]],
        [-1, 0, 1], format='csc')

    # RHS: (I + (1-theta)*L)
    B = sp.diags(
        [(1.0 - theta) * (alpha - beta) * ones[:-1],
         (1.0 - (1.0 - theta) * (2.0*alpha + r_dt)) * ones,
         (1.0 - theta) * (alpha + beta) * ones[:-1]],
        [-1, 0, 1], format='csc')

    # Condizione iniziale: payoff 1_{x<0}
    V = np.where(x < 0.0, 1.0, 0.0)

    # Time stepping (da tau=0 a tau=T)
    for n in range(N):
        # Boundary values
        bc_old = np.exp(-r * n * dt) if n > 0 else 1.0    # V[0] al passo n
        bc_new = np.exp(-r * (n + 1) * dt)                 # V[0] al passo n+1

        # RHS = B * V_inner + correzioni bordo
        rhs = B.dot(V[1:-1])

        # Correzione bordo sinistro (bordo destro = 0, nessun contributo)
        rhs[0] += theta * (alpha - beta) * bc_new \
                + (1.0 - theta) * (alpha - beta) * bc_old

        # Risolvi sistema lineare
        V[1:-1] = spsolve(A, rhs)
        V[0]  = bc_new
        V[-1] = 0.0

    return x, V

## 4. Profilo del prezzo V(S)

In [ ]:
M0, N0 = 500, 500

x_cn, V_cn = solve_digital_put(r, sigma, K, T, M0, N0, theta=0.5)
x_ie, V_ie = solve_digital_put(r, sigma, K, T, M0, N0, theta=1.0)

# Soluzione analitica su griglia
S_grid = K * np.exp(x_cn)
d2_grid = (x_cn + (r - 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
V_analytic = np.exp(-r*T) * norm.cdf(-d2_grid)

plt.figure(figsize=(10, 5))
plt.plot(S_grid, V_cn, 'b-', lw=1.5, label='Crank-Nicolson')
plt.plot(S_grid, V_ie, 'r--', lw=1.5, label='Eulero Implicito')
plt.plot(S_grid, V_analytic, 'k:', lw=1, label='Analitico')
plt.xlim(0, 3)
plt.xlabel('S'); plt.ylabel('V(S, 0)')
plt.title(f'Digital Put — M={M0}, N={N0}')
plt.axvline(K, color='gray', ls=':', alpha=0.5, label='S = K')
plt.legend(); plt.grid(alpha=0.3)
plt.show()

## 5. Convergenza ($M = N$ raddoppiano insieme)

In [ ]:
grids = [50, 100, 200, 400, 800, 1600]
x0 = np.log(S0 / K)  # = 0

pr_cn, pr_ie = [], []
err_cn, err_ie = [], []

for M in grids:
    xc, Vc = solve_digital_put(r, sigma, K, T, M, M, theta=0.5)
    xi, Vi = solve_digital_put(r, sigma, K, T, M, M, theta=1.0)

    vc = np.interp(x0, xc, Vc)
    vi = np.interp(x0, xi, Vi)

    pr_cn.append(vc); pr_ie.append(vi)
    err_cn.append(abs(vc - V_exact))
    err_ie.append(abs(vi - V_exact))

## 6. Tabella riassuntiva

In [ ]:
# Calcolo ratio e ordine
ratio_cn = [np.nan] + [err_cn[i-1]/err_cn[i] for i in range(1, len(grids))]
ratio_ie = [np.nan] + [err_ie[i-1]/err_ie[i] for i in range(1, len(grids))]
p_cn = [np.nan] + [np.log2(ratio_cn[i]) if ratio_cn[i]>0 else np.nan for i in range(1, len(grids))]
p_ie = [np.nan] + [np.log2(ratio_ie[i]) if ratio_ie[i]>0 else np.nan for i in range(1, len(grids))]

df = pd.DataFrame({
    'M=N': grids,
    'Prezzo CN': pr_cn,
    'Errore CN': err_cn,
    'Ratio CN': ratio_cn,
    'p CN': p_cn,
    'Prezzo IE': pr_ie,
    'Errore IE': err_ie,
    'Ratio IE': ratio_ie,
    'p IE': p_ie,
})

print(f"Prezzo analitico: {V_exact:.10f}")
print(f"Parametri: r={r}, sigma={sigma}, S0={S0}, K={K}, T={T}\n")

# Formattazione tabella
fmt = {
    'Prezzo CN': '{:.8f}'.format, 'Prezzo IE': '{:.8f}'.format,
    'Errore CN': '{:.2e}'.format, 'Errore IE': '{:.2e}'.format,
    'Ratio CN': '{:.4f}'.format, 'Ratio IE': '{:.4f}'.format,
    'p CN': '{:.3f}'.format, 'p IE': '{:.3f}'.format,
}
print(df.to_string(index=False, formatters=fmt, na_rep='-'))

## 7. Grafici di convergenza

In [ ]:
gs = np.array(grids, dtype=float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Errore log-log
ax1.loglog(gs, err_cn, 'bo-', label='Crank-Nicolson')
ax1.loglog(gs, err_ie, 'rs-', label='Eulero Implicito')
ax1.loglog(gs, err_ie[0]*(gs[0]/gs)**1, 'k:', alpha=0.3, label='$O(h)$')
ax1.loglog(gs, err_cn[0]*(gs[0]/gs)**2, 'k--', alpha=0.3, label='$O(h^2)$')
ax1.set_xlabel('M = N'); ax1.set_ylabel('|Errore|')
ax1.set_title('Convergenza (log-log)')
ax1.legend(); ax1.grid(True, which='both', alpha=0.3)

# Prezzo
ax2.plot(gs, pr_cn, 'bo-', label='CN')
ax2.plot(gs, pr_ie, 'rs-', label='IE')
ax2.axhline(V_exact, color='k', ls='--', label=f'Analitico = {V_exact:.6f}')
ax2.set_xlabel('M = N'); ax2.set_ylabel('Prezzo V(S₀, 0)')
ax2.set_title('Convergenza del prezzo')
ax2.legend(); ax2.grid(alpha=0.3)

plt.tight_layout(); plt.show()

## 8. Convergenza separata: spazio vs tempo

In [ ]:
vals = [50, 100, 200, 400, 800, 1600]

# Convergenza in SPAZIO (N=2000 fisso)
e_cn_s, e_ie_s = [], []
for M in vals:
    xg, Vc = solve_digital_put(r, sigma, K, T, M, 2000, 0.5)
    _,  Vi = solve_digital_put(r, sigma, K, T, M, 2000, 1.0)
    e_cn_s.append(abs(np.interp(x0, xg, Vc) - V_exact))
    e_ie_s.append(abs(np.interp(x0, xg, Vi) - V_exact))

# Convergenza in TEMPO (M=2000 fisso)
e_cn_t, e_ie_t = [], []
for N in vals:
    xg, Vc = solve_digital_put(r, sigma, K, T, 2000, N, 0.5)
    _,  Vi = solve_digital_put(r, sigma, K, T, 2000, N, 1.0)
    e_cn_t.append(abs(np.interp(x0, xg, Vc) - V_exact))
    e_ie_t.append(abs(np.interp(x0, xg, Vi) - V_exact))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.loglog(vals, e_cn_s, 'bo-', label='CN')
ax1.loglog(vals, e_ie_s, 'rs-', label='IE')
ax1.set_xlabel('M (punti spazio)'); ax1.set_ylabel('|Errore|')
ax1.set_title('Conv. in spazio (N = 2000 fisso)')
ax1.legend(); ax1.grid(True, which='both', alpha=0.3)

ax2.loglog(vals, e_cn_t, 'bo-', label='CN')
ax2.loglog(vals, e_ie_t, 'rs-', label='IE')
ax2.set_xlabel('N (passi tempo)'); ax2.set_ylabel('|Errore|')
ax2.set_title('Conv. in tempo (M = 2000 fisso)')
ax2.legend(); ax2.grid(True, which='both', alpha=0.3)

plt.tight_layout(); plt.show()

## 9. Commenti

**Cosa osserviamo dalla tabella e dai grafici:**

1. **Entrambi i metodi convergono con ratio ≈ 2** (ordine $p \approx 1$), anche se Crank-Nicolson dovrebbe teoricamente avere ordine 2 in tempo. Questo succede perché il payoff $\mathbb{1}_{S<K}$ ha una **discontinuità a salto** in $S = K$: la discontinuità genera oscillazioni numeriche nei primi passi di CN che ne degradano l'ordine di convergenza.

2. **Eulero Implicito** è più dissipativo, quindi smorza le oscillazioni ma è intrinsecamente ordine 1 in tempo.

3. **Crank-Nicolson** produce piccole oscillazioni vicino a $S = K$ (visibili nel grafico del profilo), che si propagano nel tempo e limitano la convergenza.

4. **Dalla convergenza separata** si vede che in **spazio** entrambi convergono bene ($\approx O(\Delta x^2)$), ma in **tempo** CN non riesce a sfruttare il suo $O(\Delta\tau^2)$ a causa della discontinuità.

5. In pratica, per payoff discontinui come la digital, **CN e IE danno risultati molto simili** a parità di griglia.

**Possibile miglioramento:** il Rannacher time-stepping (primi 2-4 passi con Eulero Implicito per smorzare le oscillazioni, poi Crank-Nicolson) permetterebbe di recuperare l'ordine 2 anche con payoff discontinui.